# Preprocesamiento
### Security Data Science -- Laboratorio 3
#### Adrian Rodriguez 21619
#### Jose Daniel Gomez Cabrera 21429

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

Cargar el dataset

In [2]:
df = pd.read_csv('./data/MalBehavD-V1-dataset.csv')

## Preprocesamiento

Convertir las secuencias de API en un formato adecuado para el análisis

In [3]:
def create_api_sequences(dataframe):
    sequences = []
    labels = []

    for i in range(dataframe.shape[0]):
        # Extraer las APIs no nulas de cada fila
        apis = dataframe.iloc[i, 2:].dropna().tolist()

        # Convertir la lista de APIs en una cadena, donde cada API está separada por un espacio
        api_sequence = ' '.join(apis)

        sequences.append(api_sequence)
        labels.append(dataframe.iloc[i, 1])  # Columna 'labels'

    return sequences, labels

In [4]:
api_sequences, labels = create_api_sequences(df)

In [5]:
print("Ejemplos de secuencias de API:")
for i in range(3):
    print(f"Muestra {i+1}:")
    print(f"Etiqueta: {labels[i]}")
    print(f"Secuencia: {api_sequences[i][:150]}...")  # Mostrar los primeros 150 caracteres
    print()

Ejemplos de secuencias de API:
Muestra 1:
Etiqueta: 0
Secuencia: LdrUnloadDll CoUninitialize NtQueryKey NtDuplicateObject GetShortPathNameW GetSystemInfo IsDebuggerPresent GetSystemWindowsDirectoryW NtClose GetFileV...

Muestra 2:
Etiqueta: 0
Secuencia: NtOpenMutant GetForegroundWindow NtQueryKey DrawTextExW NtSetInformationFile RegQueryValueExA LdrGetProcedureAddress CoUninitialize NtQueryValueKey Re...

Muestra 3:
Etiqueta: 0
Secuencia: GetForegroundWindow DrawTextExW GetSystemInfo IsDebuggerPresent GetSystemWindowsDirectoryW NtQueryValueKey RegCloseKey GetFileAttributesW RegQueryValu...



## Implementación y validación (70% entrenamiento y 30 pruebas)

In [6]:
# División de datos en conjuntos de entrenamiento y prueba (70/30)
X_train, X_test, y_train, y_test = train_test_split(
    api_sequences, labels, test_size=0.3, random_state=42, stratify=labels
)

In [7]:
print(f"Cantidad de muestras de entrenamiento: {len(X_train)}")
print(f"Cantidad de muestras de prueba: {len(X_test)}")

Cantidad de muestras de entrenamiento: 1799
Cantidad de muestras de prueba: 771


## Ingeniería de Características con BoW (Bag of Words)

In [8]:
bow_vectorizer = CountVectorizer(ngram_range=(1, 1))  # Unigramas
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

In [9]:
print(f"Dimensiones de X_train_bow: {X_train_bow.shape}")
print(f"Cantidad de características (APIs únicas): {len(bow_vectorizer.get_feature_names_out())}")

Dimensiones de X_train_bow: (1799, 286)
Cantidad de características (APIs únicas): 286


## Ingeniería de Características con TF-IDF

In [10]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))  # Unigramas y bigramas
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [11]:
print(f"Dimensiones de X_train_tfidf: {X_train_tfidf.shape}")
print(f"Cantidad de características (unigramas y bigramas únicos): {len(tfidf_vectorizer.get_feature_names_out())}")

Dimensiones de X_train_tfidf: (1799, 6754)
Cantidad de características (unigramas y bigramas únicos): 6754


n-gramas más importantes según el valor TF-IDF

In [12]:
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_sums = X_train_tfidf.sum(axis=0).A1
top_indices = tfidf_sums.argsort()[-20:]  # Top 20 n-gramas con mayor valor TF-IDF

In [13]:
print("\nTop 20 n-gramas más relevantes según el valor de TF-IDF:")
for idx in top_indices:
    print(f"{feature_names[idx]}: {tfidf_sums[idx]:.4f}")


Top 20 n-gramas más relevantes según el valor de TF-IDF:
ntcreatefile ntclose: 56.3180
ntqueryvaluekey ldrunloaddll: 57.8926
createprocessinternalw: 57.9778
ntopenmutant: 58.7904
getfilesize: 59.3395
getsystemtimeasfiletime: 59.3417
ldrgetdllhandle: 60.1386
ntfreevirtualmemory: 60.1740
ntprotectvirtualmemory: 60.5715
ntwritefile: 60.7630
ntreadfile: 61.6069
regclosekey: 62.6073
ntallocatevirtualmemory: 64.5910
ntcreatefile: 65.5279
ldrgetprocedureaddress: 67.6497
ldrunloaddll: 69.2425
ntqueryvaluekey: 70.0655
ntopenkey: 70.0685
ntclose: 70.5055
ntterminateprocess: 74.2047


Comprobar la distribución de clases en los conjuntos de entrenamiento y prueba

In [14]:
print("Distribución de clases:")
print(f"Entrenamiento: {pd.Series(y_train).value_counts()}")
print(f"Prueba: {pd.Series(y_test).value_counts()}")

Distribución de clases:
Entrenamiento: 1    900
0    899
Name: count, dtype: int64
Prueba: 0    386
1    385
Name: count, dtype: int64


Guardar vectorizadores

In [15]:
pickle.dump(bow_vectorizer, open('./vectorizers/bow_vectorizer.pkl', 'wb'))
pickle.dump(tfidf_vectorizer, open('./vectorizers/tfidf_vectorizer.pkl', 'wb'))

Guardar los preprocesadores

In [16]:
np.save('./preprocessors/X_train_bow.npy', X_train_bow.toarray())
np.save('./preprocessors/X_test_bow.npy', X_test_bow.toarray())
np.save('./preprocessors/X_train_tfidf.npy', X_train_tfidf.toarray())
np.save('./preprocessors/X_test_tfidf.npy', X_test_tfidf.toarray())
np.save('./preprocessors/y_train.npy', np.array(y_train))
np.save('./preprocessors/y_test.npy', np.array(y_test))